In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
data = pd.read_csv('DATA_PROVINSI.csv')

df = pd.DataFrame(data)

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Menambahkan kolom lag (pergeseran)
df['jumlah_kecelakaan_lag'] = df['jumlah_kecelakaan'].shift(1)
df = df.dropna() # Menghapus baris dengan nilai NaN

In [3]:
# Memisahkan data latih dan data uji sesuai rasio (0.8 untuk 80:20)
train_size = int(0.8 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Memisahkan fitur dan target untuk data latih
X_train = train_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_train = train_data['jumlah_kecelakaan'].values

# Memisahkan fitur dan target untuk data uji
X_test = test_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_test = test_data['jumlah_kecelakaan'].values

In [4]:
# Define evaluation function
def evaluate(individual):
    C, epsilon = individual
    if C <= 0 or epsilon <= 0 :
        return float('inf'),
    svr = SVR(kernel='linear', C=C, epsilon=epsilon)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return mape,
    
# Setup DEAP framework
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Parameter ranges
C_range = (0.01, 5)
epsilon_range = (0.001, 5)

def create_individual():
    C = random.uniform(*C_range)
    epsilon = random.uniform(*epsilon_range)
    return creator.Individual([C, epsilon])
    
toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxUniform, indpb=0.5)  # Menggunakan crossover uniform 
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2) # Menggunakan mutasi gaussian
toolbox.register("select", tools.selTournament, tournsize=3) # Menggunakan seleksi turnamen
toolbox.register("evaluate", evaluate)

In [5]:
# Initialize population
population_size = 300
population = toolbox.population(n=population_size)

# Genetic algorithm parameters    
num_generations = 80
crossover_rate = 0.2
mutation_rate = 0.8

# Run genetic algorithm
for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)
    for ind in offspring:
        ind[0] = max(min(ind[0], C_range[1]), C_range[0])
        ind[1] = max(min(ind[1], epsilon_range[1]), epsilon_range[0])
        
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
        print(f"Individu: {ind}, MAPE: {fit[0]}")
    
    population = toolbox.select(offspring, k=len(population))

# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
best_mape = best_individual.fitness.values[0]
C, epsilon = best_individual

# Test the best SVR model
best_svr = SVR(kernel='linear', C=C, epsilon=epsilon)
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print("\nParameter terbaik:")
print(f"C: {C}")
print(f"Epsilon: {epsilon}")
print(f"MAPE: {mape:.4f}")

Generasi 1:
Individu: [0.9835008265975616, 4.259084750568258], MAPE: 20.390900629871226
Individu: [1.2423080624553136, 1.409394581768768], MAPE: 20.390900629871226
Individu: [3.306435665734607, 1.8264024220738126], MAPE: 20.390900629871226
Individu: [1.4076121002425195, 3.839826701495504], MAPE: 20.390900629871226
Individu: [1.6874646998950034, 2.1796646046231567], MAPE: 20.390900629871226
Individu: [0.41276226668328436, 1.5185902333057602], MAPE: 20.390900629871226
Individu: [1.3398382002471272, 2.8769231089656246], MAPE: 20.390900629871226
Individu: [0.9992619783178156, 4.133752139694979], MAPE: 20.390900629871226
Individu: [4.193779824621857, 1.8238531816281551], MAPE: 20.390900629871226
Individu: [0.5253081105329233, 2.1842571894396654], MAPE: 20.390900629871226
Individu: [2.906940300345183, 2.616286351404877], MAPE: 20.390900629871226
Individu: [4.2235709176421805, 2.965571385119426], MAPE: 20.390900629871226
Individu: [2.0148868002004545, 2.9036417338084144], MAPE: 20.39090062987

In [6]:
# Prediksi 6 bulan ke depan
last_value = df['jumlah_kecelakaan'].values[-1] # Nilai terakhir dari data sebenarnya
predicts = []

for i in range(6):
    next_value = best_svr.predict(np.array([[last_value]]))[0]
    predicts.append(next_value)
    last_value = next_value

# Denormalisasi prediksi
predicts = scaler.inverse_transform(np.array(predicts).reshape(-1, 1))

print("Prediksi untuk 6 bulan ke depan:")
print(predicts)

Prediksi untuk 6 bulan ke depan:
[[18.21246678]
 [18.15769224]
 [18.15388256]
 [18.15361759]
 [18.15359916]
 [18.15359788]]
